### Базы данных
#### Ссылки:
  * https://fastapi.tiangolo.com/tutorial/sql-databases/
  * https://sqlmodel.tiangolo.com/tutorial/fastapi/
  * https://blog.miguelgrinberg.com/post/the-flask-mega-tutorial-part-iv-database
  * https://docs.sqlalchemy.org/en/14/orm/tutorial.html
  * https://docs.sqlalchemy.org/en/14/orm/quickstart.html
  * https://fastapi.tiangolo.com/tutorial/security/oauth2-jwt/#check-it

#### Для чего они нужны
На прошедших семинарах программы брали пользовательский ввод с клавиатуры, выводили на экран, и никаким образом долговременно результаты работы программы не сохранялись. Но необходимость надежно хранить данные и уметь в них ориентироваться жизненно важна для современного состояния дел в индустрии.
  * Учетная информация (имена пользователей и пароли)
  * История покупок
  * Складские запасы
  * Результаты опросов, наблюдений и экспериментов
  * Еда и котики в Инстаграме* (Мета признана экстремистской организацией)

Для того, чтобы надежно сохранять данные во время и после завершения работы программы, придуманы ПРОСТЫЕ СОВЕТСКИЕ... файлы на жестком диске, ничего другого в природе операционные системы нам не предоставлют. НУЖНО ВСЕГО ЛИШЬ взять и записать данные в файл

```python
with open('file.txt', 'w') as f:
    for result in results:
        f.write(str(result))
        f.write('\n')
```
Какие сложности здесь имеются
  * Писать в файл в один момент времени может только один процесс
  * Файл может исчезнуть вместе с диском
  * Поиск по файлу в общем случае линейный
  * Запись данных может прерваться посередине процесса по самым разным причинам: что-то запишется, а что-то нет
  * Данные доступны на диске только одного какого-то компьютера, и если он недоступен по сети - то и они не доступны.

Для того, чтобы эти сложности решить, и позволить разработчику ПО сосредоточиться собственно на разработке прикладного ПО, существует класс систем: СУБД - системы управления базами данных. В их основе по-прежнему запись байтов на диски по разным файлам, ничего другого человечество не придумано, но кроме этого встроены механизмы параллельной работы, надежной записи, хранения и доступности, целостности изменений, быстрого поиска и обновления.
Как это все устроено, вам *возможно* расскажут на дальнейших курсах, а в рамках этого семинара разберем основные понятия, и как этим всем пользоваться.

#### Реляционнные и нереляционные БД
В основе реляционных баз данных лежит следующая идея:
  * Данные одного типа опишем в едином формате и поместим в одно место (таблицу)
  * Связи между данными из разных доменов опишем в виде связей между таблицами (поле одной таблицы можно связать с полем другой таблицы)
  * Предоставим язык для описания данных и для поиска/изменения их, который не зависит от того, как именно реализована база данных. (Structured Query Language - SQL)

В основе нереляционных баз данных лежит другая идея - давайте ничего этого не делать. Обеспечим надежность хранения, дадим какой-то язык (стандарта как такового нет), а связи между данными уже пользователь обеспечит сам, если они ему нужны.
Сами данные при этом могут лежать в практически произвольном формате (например ключ: произвольный json)

Далее мы будем рассматривать реляционные базы данных.

#### Транзакции
Транзакция это набор из одной или нескольких операций с данными в базе.
Нужно запомнить аббревиатуру ACID - 4 основных свойства транзакции Atomicity, Consistency, Isolation, Durability

  * Atomicity - Атомарность. Транзакция должна быть выполнена целиком или не выполняться вообще.
  * Consistency - Согласованность. Транзакция не должна нарушать целостность данных.
  * Isolation - Изоляция. Транзакции не должны влиять друг на друга.
  * Durability - Длительность. Изменения должны быть сохранены после завершения транзакции.

 #### SQL, DDL и DDL
   * SQL - Structured Query Language. DDL - Data definition language. Описывает таблицы, связи между ними, и другие объекты базы данных: триггеры, view и хранимые процедуры, (которые нам не понадобятся в рамках этого семинара)
   * DML - Data manipulation language. Описывает операции с данными: вставка, удаление, обновление.

   ```sql
   -- Это комментарий
   -- Создать таблицу users с полями id, name, password_hash - DDL
   CREATE TABLE users(
    id INT NOT NULL AUTO_INCREMENT, -- в общем случае так делать не надо, но в учебных целях будет достаточно
    name VARCHAR(255) NOT NULL,
    password_hash VARCHAR(255) NOT NULL,
    PRIMARY KEY(id) -- в таблице может быть не более одной записи с заданной комбинацией полей первичного ключа
   );
   
   CREATE TABLE posts(
    id INT NOT NULL AUTO_INCREMENT,
    user_id INT NOT NULL,
    text VARCHAR(255) NOT NULL,
    PRIMARY KEY(id),
    FOREIGN KEY(user_id) REFERENCES users(id) -- поле user_id ссылается на поле id таблицы users. Посты лолжны принадлежать существующему пользователю
    ON DELETE CASCADE ON UPDATE CASCADE -- если пользователь удален, то все его посты также удалятся. Если же у пользователя изменится id (что крайне маловероятно), то посты, провязаннные со старым user_id, будут провязаны с новым user_id
   );
   -- Работа с таблицей - DML
   -- Регистрация пользователя. id возьмется последний незанятый.  
   -- c8fed00eb2e87f1cee8e90ebbe870c190ac3848c - это sha1 от `passwd`. Пароли открытым текстом в базе не храним, чтобы обезопасить себя от взлома.
   INSERT INTO users(name, password_hash) VALUES ('user1', 'c8fed00eb2e87f1cee8e90ebbe870c190ac3848c')); 
   SELECT * FROM users; -- посмотреть всех пользователей
   SELECT * FROM users WHERE name = 'user1'; -- посмотреть пользователя user1
   UPDATE users SET password_hash = 'c8fed00eb2e87f1cee8e90ebbe870c190ac3848c' WHERE name = 'user1'; -- изменить пароль пользователя user1
   -- установить новый password_hash для всех пользователей
   UPDATE users SET password_hash = 'c8fed00eb2e87f1cee8e90ebbe870c190ac3848c'
   DELETE FROM users WHERE name = 'user1'; -- удалить пользователя user1
   DELETE FROM users; -- удалить всех пользователей

   SELECT posts.* FROM posts INNER JOIN users ON posts.user_id = users.id WHERE users.name = 'user1'; -- посмотреть все посты пользователя user1

   SELECT users.id, posts.text FROM posts LEFT JOIN users ON posts.user_id = users.id WHERE posts.text is NULL;
   -- найти пользователей без постов
```

Написать `DELETE FROM users` или `UPDATE table set value чему-нибудь` без `WHERE` - мина, на которую рано или поздно наступит каждый, кто работает с базами данных, поэтому блок `WHERE` нужно писать первым делом

#### NULL
Особый маркер незаданного значения в базе данных, в общем случае для любого типа данных. Особенность его в том, что он одновременно и не равен и не неравен любому значению, и NULL в том числе
выражения `NULL = NULL` или `NULL!=NULL` равны ни `true`, ни `false`, а `NULL`

Правильно сравнивать `where field is NULL` или `where field is not NULL`.

#### Виды Join
  * **INNER JOIN** - объединение двух таблиц по полю, которое есть в каждой из двух таблиц.
  * **LEFT JOIN** - объединение двух таблиц по полю, которое есть в первой таблице.
  * **RIGHT JOIN** - объединение двух таблиц по полю, которое есть во второй таблице. Используется редко, потому что вместо него можно использовать LEFT JOIN, если поменять порядок таблиц
  * **FULL OUTER JOIN** - объединение двух таблиц по полю, которое есть хотя бы в одной из двух таблиц.

```sql
-- выведет имя пользователя и тексты его постов, если хотя бы один пост он написал
SELECT users.name, posts.text FROM users INNER JOIN posts ON users.id = posts.user_id;
-- выведет имя пользователя и тексты его постов. Если пользователь не написал ни одного поста, то он также будет выведен, в качестве posts.text будет NULL
SELECT users.name, posts.text FROM users LEFT JOIN posts ON users.id = posts.user_id;

-- выведет имя пользователя и тексты его постов. Если пользователь не написал ни одного поста, то он также будет выведен, в качестве posts.text будет NULL. Если бы были посты с неизвестным user_id, то они также будут выведены, в качестве users.name будет NULL.

SELECT users.name, posts.text FROM users FULL OUTER JOIN posts ON users.id = posts.user_id;
```

В нашем модельном примере предполагается, что поля, по которым осуществляется JOIN, NOT NULL. Если бы это было не так, и нашелся бы user с `id is NULL` и нашелся бы пост, с `user_id is NULL` то, несмотря на то, что `posts.user_id` и `users.id` равны чему-то одному и тому же (вроде бы), в выборку они не попадут, потому что эти два NULL не сравнимы.

**Совет**
  * Проектируйте схему данных (таблицы и связи между ними) таким образом, чтобы поля, по которым осуществляется JOIN, были NOT NULL.

#### Индексы
Индексы - специальные структуры данных, которые позволяют быстро искать данные в базе данных с целью показа/обновления/удаления данных. Без индексов для того, чтобы найти подходящие под условия запроса данные, нужно просканировать все таблицы, по которым идет запрос. Это явление называется `Full Table Scan`. Кроме того, индексы могут обеспечить уникальность колонок, не входящих в первичный ключ.
```sql
-- Быстро можем найти пользователя по имени и запретить появление нескольких пользователей с одинаковым именем
CREATE UNIQUE INDEX IF NOT EXISTS name_index ON users (name);
```


#### ORM

На заре индустрии работа приложений с базами данных заключалась в следующем: На все случаи жизни были заготовлены явно написанные SQL запросы, например.

```python
def get_user_by_id(id):
    cursor = connection.cursor()
    # на примере ниже - SQL инъекция, так не делайте
    cursor.execute("SELECT * FROM users WHERE id = %s", (id,))
    # а так безопасно, потому что ? - это параметр запроса, движок не позволит подсунуть туда произвольный текст
    cursor.execute("SELECT * FROM users WHERE id = ?", id)
    return cursor.fetchone()
```
В некоторых экосистемах/языках программирования до сих пор считается, что ровно так и нужно работать с базами данных. Все запросы пишем сами, никаких неожиданностей, что написали - то и исполняется, есть возможность оптимизации сложных запросов.  Есть другой подход - ORM (Object Relational Mapping)

Размечаем поля классов полями таблиц, а дальше случается магия, и все само преобразуется куда надо, запросы тоже генерируются сами, оставляя возможность написать что-то руками самому, если без этого никак. В Python самым популярным ORM является SQLAlchemy

```python
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import relationship

class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String(255), nullable=False)
    password_hash = Column(String(255), nullable=False)
    posts = relationship('Post', backref='user', lazy=True)


# session - объект, который создается при подключении к базе данных. Далее разберем пример под ключ
def get_user_by_id(id):
    # one() - возвращает ровно одну запись, если записей несколько или вообще нет, то выбрасывает исключение
    return session.query(User).filter(User.id == id).one()

def get_posts_by_user_id(user_id):
    return session.query(Post).filter(Post.user_id == user_id).all()

def get_posts_by_user_name(user_name):
    return session.query(Post).join(User).filter(User.name == user).all()
```

#### FastAPI, SQLModel, SQLAlchemy, Pydantic - соединяем все вместе

```python
from typing import Annotated

from fastapi import Depends
from fastapi import FastAPI
from fastapi import HTTPException
from fastapi import Query
from fastapi import Response
from fastapi import status
from sqlmodel import Field
from sqlmodel import SQLModel
from sqlmodel import Session
from sqlmodel import create_engine
from sqlmodel import select


# table=True - есть таблица в базе данных, герой лежит в ней
class Hero(SQLModel, table=True):
    id: int | None = Field(default=None, primary_key=True)
    name: str = Field(index=True)
    age: int | None = Field(default=None, index=True)
    secret_name: str


# будем работать с sqlite - база данных поверх локального файла
sqlite_file_name = "database.db"
sqlite_url = f"sqlite:///{sqlite_file_name}"

# Отключаем проверку того, что обращения к БД идут из одного потока (на каждый запрос создается новая сессия, поэтому они не будут друг другу мешать)
connect_args = {"check_same_thread": False}
engine = create_engine(sqlite_url, connect_args=connect_args)


def create_db_and_tables():
    # на основании разметки классов понятно, какие таблицы с какими полями нужно создать
    SQLModel.metadata.create_all(engine)


def get_session():
    with Session(engine) as session:
        yield session


# Разметка, чтобы каждый запрос обрабатывался в рамках новой сессии. Особая уличная магия
SessionDep = Annotated[Session, Depends(get_session)]

app = FastAPI()


@app.on_event("startup")
def on_startup():
    create_db_and_tables()


@app.post("/heroes/")
def create_hero(hero: Hero, session: SessionDep) -> Hero:
    session.add(hero)
    session.commit()
    session.refresh(hero)
    return hero


@app.get("/heroes/")
def read_heroes(
        session: SessionDep,
        offset: int = 0,
        limit: Annotated[int, Query(le=100)] = 100,
) -> list[Hero]:
    heroes = session.exec(select(Hero).offset(offset).limit(limit)).all()
    return heroes


@app.get("/heroes/{hero_id}")
def read_hero(hero_id: int, session: SessionDep) -> Hero:
    hero = session.get(Hero, hero_id)
    if not hero:
        raise HTTPException(status_code=404, detail="Hero not found")
    return hero


@app.delete("/heroes/{hero_id}")
def delete_hero(hero_id: int, session: SessionDep, response: Response):
    hero = session.get(Hero, hero_id)
    if not hero:
        raise HTTPException(status_code=404, detail="Hero not found")
    session.delete(hero)
    session.commit()
    response.status_code = status.HTTP_204_NO_CONTENT
    return {"ok": True}

```
